In [3]:
import os
import json
import argparse
import logging
import pickle
import random

import pandas as pd
import torch
from torch.utils import data
import lightning.pytorch as pl
from lightning.pytorch.loggers.csv_logs import CSVLogger

from proteinclip import data_utils, fasta_utils, swissprot, hparams
from proteinclip import contrastive

Loading checkpoint shards: 100%|██████████| 2/2 [00:18<00:00,  9.17s/it]
Device set to use cuda:0


In [2]:
os.chdir(os.path.join(os.getcwd(), '../'))

In [6]:
struct = pd.read_pickle('../structural_embeddings_0_2000.pkl')

In [5]:
df1 = pd.read_pickle('../data/structural/structural_embeddings_0_2000.pkl')
df2 = pd.read_parquet('../data/structural/structural_embeddings_2000_5000.parquet')
df3 = pd.read_pickle('../data/structural/structural_embeddings_5000_7500.pkl')

In [6]:
df_comb = pd.concat([df1,df2,df3])

In [11]:
df_comb.to_pickle('../data/structural/structural_embeddings_0_7500.pkl')

In [ ]:
args = {
    'structural_embed':'../data/structural/structural_embeddings_0_7500.pkl',
    'proteinclip_embed':'protclip_embed_dataset.parquet',
    'unitnorm': True,
    'batch_size': 512,
    'dim': 128,
    'nhidden': 1,
    'learning_rate':1e-4,
    'out':'training',
    'name':'testrun',
    'max_epochs': 1000,
    
}

In [16]:
pl.seed_everything(seed=6489)


[rank: 0] Seed set to 6489


6489

In [17]:
structural_embeddings = None
with open(args['structural_embed'], "rb") as f:
    structural_embeddings = pickle.load(f)
structural_embeddings = dict(zip(
    structural_embeddings["protein_id"],
    structural_embeddings["embedding"]
))

In [18]:
proteinclip_embeddings = pd.read_parquet(args['proteinclip_embed'])
proteinclip_embeddings = dict(zip(
    proteinclip_embeddings["id"],
    proteinclip_embeddings["proteinclip_embed"]
))

In [19]:
shared_keys = sorted(set(proteinclip_embeddings.keys()).intersection(set(structural_embeddings.keys())))


In [20]:
random.seed(42)  # Set your desired seed here
random.shuffle(shared_keys)


In [21]:
dset = data_utils.CLIPDataset(
    pairs=shared_keys,
    map1=proteinclip_embeddings,
    map2=structural_embeddings,
    enforce_unit_norm=args['unitnorm'],
)

Checking for missing/zero embeddings: 100%|██████████| 7493/7493 [00:00<00:00, 31490.33it/s]
INFO:root:Trimmed 0 pairs with missing/zero embeddings, 7493 remain.


In [22]:
split_indices = data_utils.random_split(len(dset), [0.9, 0.05, 0.05])
dset_splits = [data.Subset(dset, idx) for idx in split_indices]

In [23]:
# Create data loaders
train_dl, valid_dl, _test_dl = [
    data.DataLoader(
        ds,
        batch_size=args['batch_size'],
        shuffle=(i == 0),
        # drop_last=(i == 0),
        num_workers=8,
        pin_memory=True,
    )
    for i, ds in enumerate(dset_splits)
]


/home/hice1/savunuri3/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [24]:
# Define network
input_dim_1 = next(iter(train_dl))["x_1"].shape[-1]
input_dim_2 = next(iter(train_dl))["x_2"].shape[-1]
model_class = (
    contrastive.ContrastiveEmbedding
)
net = model_class(
    input_dim_1=input_dim_1,
    input_dim_2=input_dim_2,
    shared_dim=args['dim'],
    num_hidden=args['nhidden'],
    lr=args['learning_rate'],
)

In [ ]:
# # Define logger, write configuration files and data splits
# logger = CSVLogger(save_dir=args['out'], name=args['name'])
# # logger.log_hyperparams(hyperparameters.as_dict())
# write_split_identifiers(
#     train_ids=[dset.pairs[i] for i in split_indices[0]],
#     valid_ids=[dset.pairs[i] for i in split_indices[1]],
#     test_ids=[dset.pairs[i] for i in split_indices[2]],
#     out_file=os.path.join(logger.log_dir, "data_splits.json"),
# )
# net.write_config_json(os.path.join(logger.log_dir, "model_config.json"))
# with open(os.path.join(logger.log_dir, "training_config.json"), "w") as sink:
#     json.dump(vars(args), sink, indent=4)


In [25]:
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
early_stop_callback = EarlyStopping(
    monitor='val_loss',         # metric name to monitor
    patience=50,                 # epochs with no improvement after which to stop
    mode='min',                 # 'min' if you want to minimize the metric
    verbose=True                # optional: prints a message when triggered
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    save_top_k=1,
    mode='min',
    filename='best-checkpoint'
)


In [26]:
# Train
trainer = pl.Trainer(
    max_epochs=1000,
    accelerator="cuda",
    devices=1,
    enable_progress_bar=True,
    # logger=logger,
    # log_every_n_steps=10,
    deterministic=True,
    callbacks=[early_stop_callback, checkpoint_callback],

)
trainer.fit(net, train_dataloaders=train_dl, val_dataloaders=valid_dl)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/hice1/savunuri3/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA H200') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_

Epoch 0: 100%|██████████| 106/106 [00:03<00:00, 26.63it/s, v_num=2509880, train_loss=1.850, val_loss=2.850]

Metric val_loss improved. New best score: 2.849


Epoch 1: 100%|██████████| 106/106 [00:00<00:00, 119.02it/s, v_num=2509880, train_loss=1.790, val_loss=2.650]

Metric val_loss improved by 0.202 >= min_delta = 0.0. New best score: 2.647


...

Epoch 312: 100%|██████████| 106/106 [00:00<00:00, 119.88it/s, v_num=2509880, train_loss=0.0194, val_loss=0.107]  

Metric val_loss improved by 0.002 >= min_delta = 0.0. New best score: 0.107


Epoch 362: 100%|██████████| 106/106 [00:00<00:00, 123.99it/s, v_num=2509880, train_loss=0.0177, val_loss=0.144]  

Monitored metric val_loss did not improve in the last 50 records. Best score: 0.107. Signaling Trainer to stop.


Epoch 362: 100%|██████████| 106/106 [00:00<00:00, 123.79it/s, v_num=2509880, train_loss=0.0177, val_loss=0.144]


In [27]:



# Export model as ONNX files
contrastive.model_to_onnx(
    net.project_1,
    "project_1_7500.onnx",
    input_shape=(input_dim_1,),
)
contrastive.model_to_onnx(
    net.project_2,
    "project_2_7500.onnx",
    input_shape=(input_dim_2,),
)


2025-04-24 11:42:48.631390698 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 3152785, index: 9, mask: {36, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-24 11:42:48.631398952 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 3152796, index: 20, mask: {10, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-24 11:42:48.631400791 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 3152800, index: 24, mask: {54, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-24 11:42:48.645556379 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 3152837, index: 61, mask: {39, }, error code: 22 error msg: Invalid argument. Specif